### Part 1

GOAL: The Army wants to meet the nutritional requirements of its soldiers while minimizing the cost. Given the list of food options, including their nutrition facts, formulate an optimization model using a linear program to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints.

Suppose we are given the following known variables:

n food

m nutrients

$a_{ij}$ = amount of nutrient j per unit of food i

$m_{j}$ = minimum daily intake of nutrient j

$M_{j}$ = minimum daily intake of nutrient j

$c_{i}$ = per-unit cost of food i

1.  **Identify the Decision Variables**

To find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, we first define our decision variable $x_{i}$, which is the amount of food i in daily diet.

$x_{1}$ = amount of Frozen Broccoli

$x_{2}$ = amount of Carrots, Raw

...

2.  **Formulate the Constraints**

The constraints are:

-   $x_{i} \ge 0$ for each food i
-   for each nutrient j:
 $m_{j} \le \sum_ia_{ij}x{i} \le M_{j}$

3.  **Formulate the Objective Function**

The objective is to minimize the total cost of food while meeting the constraints. We know that cost per serving of each ingredient.

min $\sum_ic_{ij}x{i}$


In [152]:
from pulp import *
import pandas as pd

Let's first load in the dataset and then create dictionaries of cost and each nutrients (e.g., carbohydrates, fiber, etc.) for each ingredient.

In [153]:
df = pd.read_excel('diet.xls', sheet_name = 'Sheet1')
df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Foods            64 non-null     object 
 1   Price/ Serving   64 non-null     float64
 2   Serving Size     64 non-null     object 
 3   Calories         64 non-null     float64
 4   Cholesterol mg   64 non-null     float64
 5   Total_Fat g      64 non-null     float64
 6   Sodium mg        64 non-null     float64
 7   Carbohydrates g  64 non-null     float64
 8   Dietary_Fiber g  64 non-null     float64
 9   Protein g        64 non-null     float64
 10  Vit_A IU         64 non-null     float64
 11  Vit_C IU         64 non-null     float64
 12  Calcium mg       64 non-null     float64
 13  Iron mg          64 non-null     float64
dtypes: float64(12), object(2)
memory usage: 7.1+ KB


,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0


In [154]:
costs = {}
calorie_dict = {}
Cholesterol_dict = {}
fat_dict = {}
sodium_dict = {}
carbohydrate_dict = {}
fiber_dict = {}
protein_dict = {}
vit_A_dict = {}
vit_C_dict = {}
calcium_dict = {}
iron_dict = {}

for index, val in df.iloc[:, 0].items():
    costs[val] = df.iloc[index, 1]
    calorie_dict[val] = df.iloc[index, 3]
    Cholesterol_dict[val] = df.iloc[index, 4]
    fat_dict[val] = df.iloc[index, 5]
    sodium_dict[val] = df.iloc[index, 6]
    carbohydrate_dict[val] = df.iloc[index, 7]
    fiber_dict[val] = df.iloc[index, 8]
    protein_dict[val] = df.iloc[index, 9]
    vit_A_dict[val] = df.iloc[index, 10]
    vit_C_dict[val] = df.iloc[index, 11]
    calcium_dict[val] = df.iloc[index, 12]
    iron_dict[val] = df.iloc[index, 13]


{'Frozen Broccoli': 0.16, 'Carrots,Raw': 0.07, 'Celery, Raw': 0.04, 'Frozen Corn': 0.18, 'Lettuce,Iceberg,Raw': 0.02, 'Peppers, Sweet, Raw': 0.53, 'Potatoes, Baked': 0.06, 'Tofu': 0.31, 'Roasted Chicken': 0.84, 'Spaghetti W/ Sauce': 0.78, 'Tomato,Red,Ripe,Raw': 0.27, 'Apple,Raw,W/Skin': 0.24, 'Banana': 0.15, 'Grapes': 0.32, 'Kiwifruit,Raw,Fresh': 0.49, 'Oranges': 0.15, 'Bagels': 0.16, 'Wheat Bread': 0.05, 'White Bread': 0.06, 'Oatmeal Cookies': 0.09, 'Apple Pie': 0.16, 'Chocolate Chip Cookies': 0.03, 'Butter,Regular': 0.05, 'Cheddar Cheese': 0.25, '3.3% Fat,Whole Milk': 0.16, '2% Lowfat Milk': 0.23, 'Skim Milk': 0.13, 'Poached Eggs': 0.08, 'Scrambled Eggs': 0.11, 'Bologna,Turkey': 0.15, 'Frankfurter, Beef': 0.27, 'Ham,Sliced,Extralean': 0.33, 'Kielbasa,Prk': 0.15, "Cap'N Crunch": 0.31, 'Cheerios': 0.28, "Corn Flks, Kellogg'S": 0.28, "Raisin Brn, Kellg'S": 0.34, 'Rice Krispies': 0.32, 'Special K': 0.38, 'Oatmeal': 0.82, 'Malt-O-Meal,Choc': 0.52, 'Pizza W/Pepperoni': 0.44, 'Taco': 0.59, 

Let's now make a list of ingredients, which are taken from the first column of the dataset.

In [156]:
# Make a list of ingredients (food)
ingredients = df["Foods"].tolist()

['Frozen Broccoli',
 'Carrots,Raw',
 'Celery, Raw',
 'Frozen Corn',
 'Lettuce,Iceberg,Raw',
 'Peppers, Sweet, Raw',
 'Potatoes, Baked',
 'Tofu',
 'Roasted Chicken',
 'Spaghetti W/ Sauce',
 'Tomato,Red,Ripe,Raw',
 'Apple,Raw,W/Skin',
 'Banana',
 'Grapes',
 'Kiwifruit,Raw,Fresh',
 'Oranges',
 'Bagels',
 'Wheat Bread',
 'White Bread',
 'Oatmeal Cookies',
 'Apple Pie',
 'Chocolate Chip Cookies',
 'Butter,Regular',
 'Cheddar Cheese',
 '3.3% Fat,Whole Milk',
 '2% Lowfat Milk',
 'Skim Milk',
 'Poached Eggs',
 'Scrambled Eggs',
 'Bologna,Turkey',
 'Frankfurter, Beef',
 'Ham,Sliced,Extralean',
 'Kielbasa,Prk',
 "Cap'N Crunch",
 'Cheerios',
 "Corn Flks, Kellogg'S",
 "Raisin Brn, Kellg'S",
 'Rice Krispies',
 'Special K',
 'Oatmeal',
 'Malt-O-Meal,Choc',
 'Pizza W/Pepperoni',
 'Taco',
 'Hamburger W/Toppings',
 'Hotdog, Plain',
 'Couscous',
 'White Rice',
 'Macaroni,Ckd',
 'Peanut Butter',
 'Pork',
 'Sardines in Oil',
 'White Tuna in Water',
 'Popcorn,Air-Popped',
 'Potato Chips,Bbqflvr',
 'Pretzel

In [157]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem('FoodProblem', LpMinimize)

Next, we need to create a variable for each food using the LpVariable class. In the following code, a dictionary called ingredient_vars is created which contains the LP (Linear Program) variables, with their defined lower bound of zero. The reference keys to the dictionary are the ingredient names, and the data is Amounts_IngredientName (e.g., Frozen Broccoli = Amounts_Frozen_Broccoli).

In [158]:
# A dictionary called 'ingredient_vars' is created to contain the referenced variables
ingredient_vars = LpVariable.dicts("Amounts", ingredients, lowBound= 0, cat = 'Continuous')

We can now use list comprehension to extract the data to build an objective function. The lpSum() function will add the elements of the resulting list.

In [159]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i] * ingredient_vars[i] for i in ingredients])

# Adding constraints
prob += (lpSum(calorie_dict[i] * ingredient_vars[i] for i in ingredients) >= 1500, 
         "min 1500 calorie")
prob += (lpSum(calorie_dict[i] * ingredient_vars[i] for i in ingredients) <= 2500, 
         "max 2500 calorie")
prob += (lpSum(Cholesterol_dict[i] * ingredient_vars[i] for i in ingredients) >= 30, 
         "min 30mg cholesterol")
prob += (lpSum(Cholesterol_dict[i] * ingredient_vars[i] for i in ingredients) <= 240, 
         "max 240mg cholesterol")
prob += (lpSum(fat_dict[i] * ingredient_vars[i] for i in ingredients) >= 20, 
         "min 20g fat")
prob += (lpSum(fat_dict[i] * ingredient_vars[i] for i in ingredients) <= 70, 
         "max 70g fat")
prob += (lpSum(sodium_dict[i] * ingredient_vars[i] for i in ingredients) >= 800, 
         "min 800mg sodium")
prob += (lpSum(sodium_dict[i] * ingredient_vars[i] for i in ingredients) <= 2000, 
         "max 2000mg sodium")
prob += (lpSum(carbohydrate_dict[i] * ingredient_vars[i] for i in ingredients) >= 130, 
         "min 130g carbohydrates")
prob += (lpSum(carbohydrate_dict[i] * ingredient_vars[i] for i in ingredients) <= 450, 
         "max 450g carbohydrates")
prob += (lpSum(fiber_dict[i] * ingredient_vars[i] for i in ingredients) >= 125, 
         "min 125g fiber")
prob += (lpSum(fiber_dict[i] * ingredient_vars[i] for i in ingredients) <= 250, 
         "max 250g fiber")
prob += (lpSum(protein_dict[i] * ingredient_vars[i] for i in ingredients) >= 60, 
         "min 60g protein")
prob += (lpSum(protein_dict[i] * ingredient_vars[i] for i in ingredients) <= 100, 
         "max 100g protein")
prob += (lpSum(vit_A_dict[i] * ingredient_vars[i] for i in ingredients) >= 1000, 
         "min 1000IU Vit A")
prob += (lpSum(vit_A_dict[i] * ingredient_vars[i] for i in ingredients) <= 10000, 
         "max 10000IU Vit A")
prob += (lpSum(vit_C_dict[i] * ingredient_vars[i] for i in ingredients) >= 400, 
         "min 400IU Vit C")
prob += (lpSum(vit_C_dict[i] * ingredient_vars[i] for i in ingredients) <= 5000, 
         "max 5000IU Vit C")
prob += (lpSum(calcium_dict[i] * ingredient_vars[i] for i in ingredients) >= 700, 
         "min 700mg calcium")
prob += (lpSum(calcium_dict[i] * ingredient_vars[i] for i in ingredients) <= 1500, 
         "max 1500mg calcium")
prob += (lpSum(iron_dict[i] * ingredient_vars[i] for i in ingredients) >= 10, 
         "min 10mg iron")
prob += (lpSum(iron_dict[i] * ingredient_vars[i] for i in ingredients) <= 40, 
         "max 40mg iron")


In [160]:
# Solve and check out the results
soln = prob.solve()

print(LpStatus[prob.status])

for v in prob.variables():
    print(f"{v.name} = {v.varValue: .2f}")

print(f"Cost: {value(prob.objective)}")

Optimal
Amounts_2%_Lowfat_Milk =  0.00
Amounts_3.3%_Fat,Whole_Milk =  0.00
Amounts_Apple,Raw,W_Skin =  0.00
Amounts_Apple_Pie =  0.00
Amounts_Bagels =  0.00
Amounts_Banana =  0.00
Amounts_Beanbacn_Soup,W_Watr =  0.00
Amounts_Bologna,Turkey =  0.00
Amounts_Butter,Regular =  0.00
Amounts_Cap'N_Crunch =  0.00
Amounts_Carrots,Raw =  0.00
Amounts_Celery,_Raw =  52.64
Amounts_Cheddar_Cheese =  0.00
Amounts_Cheerios =  0.00
Amounts_Chicknoodl_Soup =  0.00
Amounts_Chocolate_Chip_Cookies =  0.00
Amounts_Corn_Flks,_Kellogg'S =  0.00
Amounts_Couscous =  0.00
Amounts_Crm_Mshrm_Soup,W_Mlk =  0.00
Amounts_Frankfurter,_Beef =  0.00
Amounts_Frozen_Broccoli =  0.26
Amounts_Frozen_Corn =  0.00
Amounts_Grapes =  0.00
Amounts_Ham,Sliced,Extralean =  0.00
Amounts_Hamburger_W_Toppings =  0.00
Amounts_Hotdog,_Plain =  0.00
Amounts_Kielbasa,Prk =  0.00
Amounts_Kiwifruit,Raw,Fresh =  0.00
Amounts_Lettuce,Iceberg,Raw =  63.99
Amounts_Macaroni,Ckd =  0.00
Amounts_Malt_O_Meal,Choc =  0.00
Amounts_New_E_Clamchwd,W

Based on the none-zero values, we observed that the recommended diet is below:

unit of servings:\
Celery,_Raw =  52.64\
Frozen_Broccoli =  0.26\
Lettuce,Iceberg,Raw =  63.99\
Poached_Eggs =  0.14\
Popcorn,Air_Popped =  13.87

### Part 2

In the following section, we will solve the same problem but with additional constraints as follows:

1. If food i is selected, min 1/10 serving must be chosen\
Under this constraint, we will define a new variable $y_{i}$\
$y_{i}$ = 1 if food i is selected, 0 if not\
If $y_{i}$ = 1, $x_{i} \ge 1/10 * serving$

2. At most one out of celery or frozen broccoli should be chosen but not both\
$y_{celery} + y_{broccoli} \le 1$

3. At least three kinds of meat/poultry/fish/eggs must be selected.\
We have identified the following as protein sources:\
    a. Roasted Chicken\
    b. Poached Eggs\
    c. Scrambled Eggs\
    d. Bologna,Turkey\
    e. Frankfurter, Beef\
    f. Ham,Sliced,Extralean\
    g. Kielbasa,Prk\
    h. Hamburger W/Toppings\
    i. Hotdog, Plain\
    j. Pork\
    k. Sardines in Oil\
    l. White Tuna in Water\
    m. Chicknoodl Soup\
    n. Splt Pea&Hamsoup\
    o. Vegetbeef Soup\
    p. Neweng Clamchwd\
    q. New E Clamchwd,W/Mlk\
    r. Beanbacn Soup,W/Watr

Therefore, for food i in the categories above, the constraint should be:\
$\sum_iy_{i} \ge 3$

In [161]:
# Create a binary variable
food_chosen = LpVariable.dicts("Chosen", ingredients, 0, 1, cat = "Integer")
food_chosen
    

{'Frozen Broccoli': Chosen_Frozen_Broccoli,
 'Carrots,Raw': Chosen_Carrots,Raw,
 'Celery, Raw': Chosen_Celery,_Raw,
 'Frozen Corn': Chosen_Frozen_Corn,
 'Lettuce,Iceberg,Raw': Chosen_Lettuce,Iceberg,Raw,
 'Peppers, Sweet, Raw': Chosen_Peppers,_Sweet,_Raw,
 'Potatoes, Baked': Chosen_Potatoes,_Baked,
 'Tofu': Chosen_Tofu,
 'Roasted Chicken': Chosen_Roasted_Chicken,
 'Spaghetti W/ Sauce': Chosen_Spaghetti_W__Sauce,
 'Tomato,Red,Ripe,Raw': Chosen_Tomato,Red,Ripe,Raw,
 'Apple,Raw,W/Skin': Chosen_Apple,Raw,W_Skin,
 'Banana': Chosen_Banana,
 'Grapes': Chosen_Grapes,
 'Kiwifruit,Raw,Fresh': Chosen_Kiwifruit,Raw,Fresh,
 'Oranges': Chosen_Oranges,
 'Bagels': Chosen_Bagels,
 'Wheat Bread': Chosen_Wheat_Bread,
 'White Bread': Chosen_White_Bread,
 'Oatmeal Cookies': Chosen_Oatmeal_Cookies,
 'Apple Pie': Chosen_Apple_Pie,
 'Chocolate Chip Cookies': Chosen_Chocolate_Chip_Cookies,
 'Butter,Regular': Chosen_Butter,Regular,
 'Cheddar Cheese': Chosen_Cheddar_Cheese,
 '3.3% Fat,Whole Milk': Chosen_3.3%_Fa

In [162]:
for food in ingredients:
    # If food_chosen == 1, then at least 1/10 serving must be chosen. If food_chosen == 0, then the linking constraints will force the amount to 
    # become 0.
    prob += ingredient_vars[food] >= food_chosen[food] * 0.1
    prob += ingredient_vars[food] <= food_chosen[food] * 1e6

# Adding additional constraints
# At most Frozen Broccoli or Celery can be chosen but not both
prob += food_chosen['Frozen Broccoli'] + food_chosen['Celery, Raw'] <= 1
# # At least three kinds of meat/poultry/fish/eggs must be selected
prob += food_chosen['Roasted Chicken'] + food_chosen['Poached Eggs'] + food_chosen['Scrambled Eggs'] +\
food_chosen['Bologna,Turkey'] + food_chosen['Frankfurter, Beef'] + food_chosen['Ham,Sliced,Extralean'] +\
food_chosen['Kielbasa,Prk'] + food_chosen['Hamburger W/Toppings'] + food_chosen['Hotdog, Plain'] +\
food_chosen['Pork'] + food_chosen['Sardines in Oil'] + food_chosen['White Tuna in Water'] +\
food_chosen['Chicknoodl Soup'] + food_chosen['Splt Pea&Hamsoup'] + food_chosen['Vegetbeef Soup'] +\
food_chosen['Neweng Clamchwd'] + food_chosen['New E Clamchwd,W/Mlk'] + food_chosen['Beanbacn Soup,W/Watr'] >= 3


In [163]:
# Solve and check out the results
soln = prob.solve()

print(LpStatus[prob.status])

for v in prob.variables():
    print(f"{v.name} = {v.varValue: .2f}")

print(f"Cost: {value(prob.objective)}")

Optimal
Amounts_2%_Lowfat_Milk =  0.00
Amounts_3.3%_Fat,Whole_Milk =  0.00
Amounts_Apple,Raw,W_Skin =  0.00
Amounts_Apple_Pie =  0.00
Amounts_Bagels =  0.00
Amounts_Banana =  0.00
Amounts_Beanbacn_Soup,W_Watr =  0.00
Amounts_Bologna,Turkey =  0.00
Amounts_Butter,Regular =  0.00
Amounts_Cap'N_Crunch =  0.00
Amounts_Carrots,Raw =  0.00
Amounts_Celery,_Raw =  42.40
Amounts_Cheddar_Cheese =  0.00
Amounts_Cheerios =  0.00
Amounts_Chicknoodl_Soup =  0.00
Amounts_Chocolate_Chip_Cookies =  0.00
Amounts_Corn_Flks,_Kellogg'S =  0.00
Amounts_Couscous =  0.00
Amounts_Crm_Mshrm_Soup,W_Mlk =  0.00
Amounts_Frankfurter,_Beef =  0.00
Amounts_Frozen_Broccoli =  0.00
Amounts_Frozen_Corn =  0.00
Amounts_Grapes =  0.00
Amounts_Ham,Sliced,Extralean =  0.00
Amounts_Hamburger_W_Toppings =  0.00
Amounts_Hotdog,_Plain =  0.00
Amounts_Kielbasa,Prk =  0.10
Amounts_Kiwifruit,Raw,Fresh =  0.00
Amounts_Lettuce,Iceberg,Raw =  82.80
Amounts_Macaroni,Ckd =  0.00
Amounts_Malt_O_Meal,Choc =  0.00
Amounts_New_E_Clamchwd,W

Based on the output above, we observed that the recommended diet is below:

unit of servings:\
Celery,_Raw =  42.40\
Lettuce,Iceberg,Raw =  82.80\
Oranges =  3.08\
Peanut_Butter =  1.94\
Poached_Eggs =  0.10\
Popcorn,Air_Popped =  13.22\
Scrambled_Eggs =  0.10